# Caso #3

## Notebook ejecutado

In [17]:
%pip install -q amplpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: C:\Users\lec12\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [18]:
from amplpy import AMPL, ampl_notebook
ampl = ampl_notebook(
    modules=["highs", "cbc", "gurobi" ], # pick from over 20 modules including most commercial and open-source solvers
    license_uuid="09f4ede2-4840-4d72-9d94-308ef2f972f7") # your license UUID (e.g., free ampl.com/ce or ampl.com/courses licenses)

Licensed to AMPL Community Edition License for <mariana.lozano.col@gmail.com>.


In [19]:
!apt-get install -y coinor-cbc
!pip install pyomo
!pip install coinor-opt
!pip install amplpy
!pip install pyomo[solvers]
!pip install pandas
!pip install matplotlib


"apt-get" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.



[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: C:\Users\lec12\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement coinor-opt (from versions: none)
ERROR: No matching distribution found for coinor-opt

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: C:\Users\lec12\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: C:\Users\lec12\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: C:\Users\lec12\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: C:\Users\lec12\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: C:\Users\lec12\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [25]:


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from amplpy import AMPL

# === 1️⃣ Cargar datos ===

clients_df = pd.read_csv('Datos/clients3.csv')
depots_df = pd.read_csv('Datos/depots3.csv')
vehicles_df = pd.read_csv('Datos/vehicles3.csv')
stations_df = pd.read_csv('Datos/stations3.csv')
tolls_df = pd.read_csv('Datos/tolls3.csv')

# === 2️⃣ Preparar conjuntos ===

D = ['MUN' + str(int(row['LocationID'])).zfill(2) for _, row in clients_df.iterrows()]
P = ['PTO01']
E = ['EST' + str(int(row['EstationID'])).zfill(2) for _, row in stations_df.iterrows()]
V = ['VEH' + str(int(row['VehicleID'])).zfill(2) for _, row in vehicles_df.iterrows()]
T = ['PEA' + str(int(row['ClientID'])).zfill(2) for _, row in tolls_df.iterrows()]
N = P + D + E + T

# === 3️⃣ Coordenadas y distancias ===

def haversine(coord1, coord2):
    R = 6371
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

coords = {}

tolls_df['BaseRate'] = tolls_df['BaseRate'].fillna(0)
tolls_df['RatePerTon'] = tolls_df['RatePerTon'].fillna(0)

for _, row in depots_df.iterrows():
    coords['PTO01'] = (row['Latitude'], row['Longitude'])

for _, row in stations_df.iterrows():
    key = 'EST' + str(int(row['EstationID'])).zfill(2)
    coords[key] = (row['Latitude'], row['Longitude'])

for _, row in tolls_df.iterrows():
    key_tolls = 'PEA' + str(int(row['ClientID'])).zfill(2)
    matching_client = clients_df[clients_df['ClientID'] == row['ClientID']]
    if not matching_client.empty:
        lat = matching_client.iloc[0]['Latitude']
        lon = matching_client.iloc[0]['Longitude']
        coords[key_tolls] = (lat, lon)


for _, row in clients_df.iterrows():
    client_id = 'MUN' + str(int(row['LocationID'])).zfill(2)
    coords[client_id] = (row['Latitude'], row['Longitude'])


dist = {}
for i in N:
    for j in N:
        if i != j:
            dist[(i, j)] = haversine(coords[i], coords[j])

# ✅✅✅ NUEVO: Definir la lista must_pass después de calcular distancias

must_pass = []
for i in N:
    for j in N:
        if i != j and i not in E and j not in E:
            # Only consider node pairs that are far apart
            if dist.get((i, j), 0) > 20:  # Only for distances > 20km
                closest_station = None
                min_distance = float('inf')
                for e in E:
                    d_ie = dist.get((i, e), 9999)
                    d_ej = dist.get((e, j), 9999)
                    # Reduce radius from 15km to 5km
                    if d_ie < 5 or d_ej < 5:  # Stations within 5km
                        if d_ie + d_ej < min_distance:
                            min_distance = d_ie + d_ej
                            closest_station = e
                if closest_station:
                    must_pass.append((i, j, closest_station))

print(f"✅ MustPassThrough generado con {len(must_pass)} entradas.")

# === 4️⃣ Escribir archivo .dat para AMPL ===

with open('vrp_data.dat', 'w') as f:
    f.write('set V := {} ;\n'.format(' '.join(V)))
    f.write('set N := {} ;\n'.format(' '.join(N)))
    f.write('set D := {} ;\n'.format(' '.join(D)))
    f.write('set P := {} ;\n'.format(' '.join(P)))
    f.write('set E := {} ;\n'.format(' '.join(E)))
    f.write('set T := {} ;\n'.format(' '.join(T)))

    f.write('param Q :=\n')
    for _, row in vehicles_df.iterrows():
        vid = 'VEH' + str(int(row['VehicleID'])).zfill(2)
        f.write(f'{vid} {row["Capacity"]}\n')
    f.write(';\n')

    f.write('param F_cap :=\n')
    consumo = 0.75  # O el valor que tengas actualizado
    for _, row in vehicles_df.iterrows():
        vid = 'VEH' + str(int(row['VehicleID'])).zfill(2)
        F_cap = row['Range'] * consumo
        f.write(f'{vid} {F_cap}\n')
    f.write(';\n')

    f.write('param demand :=\n')
    for _, row in clients_df.iterrows():
        cid = 'MUN' + str(int(row['LocationID'])).zfill(2)
        f.write(f'{cid} {row["Demand"]}\n')
    f.write(';\n')

    f.write('param fuel_price :=\n')
    for _, row in stations_df.iterrows():
        eid = 'EST' + str(int(row['EstationID'])).zfill(2)
        f.write(f'{eid} {row["FuelCost"]}\n')
    f.write(f'PTO01 13500.0\n')
    for cid in D:
        f.write(f'{cid} 0.0\n')
    for tid in T:
        f.write(f'{tid} 0.0\n')
    f.write(';\n')

    # Nuevos parámetros para peajes
    f.write('param toll_base_rate :=\n')
    for _, row in tolls_df.iterrows():
        tid = 'PEA' + str(int(row['ClientID'])).zfill(2)
        base_rate = row['BaseRate'] if pd.notna(row['BaseRate']) else 0
        f.write(f'{tid} {base_rate}\n')
    f.write(';\n')

    f.write('param toll_rate_per_ton :=\n')
    for _, row in tolls_df.iterrows():
        tid = 'PEA' + str(int(row['ClientID'])).zfill(2)
        rate_per_ton = row['RatePerTon'] if pd.notna(row['RatePerTon']) else 0
        f.write(f'{tid} {rate_per_ton}\n')
    f.write(';\n')

    # Peso de vehículos (convertido a toneladas)
    f.write('param weight :=\n')
    for _, row in vehicles_df.iterrows():
        vid = 'VEH' + str(int(row['VehicleID'])).zfill(2)
        # Asumiendo que el peso está en kg y lo convertimos a toneladas
        weight_tons = row['Weight'] / 1000 if 'Weight' in vehicles_df.columns else row['VehicleID'] * 5  # Peso estimado si no existe
        f.write(f'{vid} {weight_tons}\n')
    f.write(';\n')

    # Restricciones de peso por municipio
    f.write('param max_weight :=\n')
    for _, row in clients_df.iterrows():
        cid = 'MUN' + str(int(row['LocationID'])).zfill(2)
        # Asumiendo que el peso máximo está en toneladas
        max_weight = row['MaxWeight'] if 'MaxWeight' in clients_df.columns else 40  # Valor por defecto
        f.write(f'{cid} {max_weight}\n')
    f.write(';\n')

    f.write('param dist :=\n')
    for (i, j), value in dist.items():
        f.write(f'{i} {j} {value:.4f}\n')
    f.write(';\n')

    # 🚀 MustPassThrough
    if must_pass:
        f.write('set MustPassThrough :=\n')
        for (i, j, e) in must_pass:
            f.write(f'({i}, {j}, {e})\n')
        f.write(';\n')
    else:
        f.write('set MustPassThrough := ;\n')

    # 🚀 Agregar parámetros escalares
    f.write('\nparam Ft := 5000;\n')
    f.write('param Cm := 700;\n')
    f.write('param consumo := 0.75;\n')

print("✅ Archivo vrp_data.dat creado correctamente!")

# === 5️⃣ Ejecutar AMPL ===

ampl = AMPL()
ampl.read('Datos/vrp_model.mod')
ampl.readData('vrp_data.dat')


ampl.setOption('solver', 'gurobi')
ampl.setOption('gurobi_options', 'TimeLimit=600')  # 600 segundos = 10 minutos
print("🚀 Resolviendo con AMPL + Gurobi (límite de 10 minutos)...")
ampl.solve()

# === 6️⃣ Procesar resultados y generar archivo de verificación ===

import csv

print("✅ Costo total global:", ampl.getObjective('TotalCost').value())

z = ampl.getVariable('z')
x = ampl.getVariable('x')
r = ampl.getVariable('r')
u = ampl.getVariable('u')
toll_used = ampl.getVariable('toll_used')

# 🚀 Cargar precios de estaciones
station_prices = {f"EST{str(int(row['EstationID'])).zfill(2)}": row['FuelCost'] for _, row in stations_df.iterrows()}

# 🚀 Cargar tarifas de peajes
toll_base_rates = {f"PEA{str(int(row['ClientID'])).zfill(2)}": row['BaseRate'] if pd.notna(row['BaseRate']) else 0 
                   for _, row in tolls_df.iterrows()}
toll_rates_per_ton = {f"PEA{str(int(row['ClientID'])).zfill(2)}": row['RatePerTon'] if pd.notna(row['RatePerTon']) else 0 
                      for _, row in tolls_df.iterrows()}

# 🚀 Cargar pesos de vehículos
vehicle_weights = {}
for _, row in vehicles_df.iterrows():
    vid = 'VEH' + str(int(row['VehicleID'])).zfill(2)
    # Asumimos que el peso está en kg y lo convertimos a toneladas
    weight_tons = row['Weight'] / 1000 if 'Weight' in vehicles_df.columns else row['VehicleID'] * 5  # Peso estimado si no existe
    vehicle_weights[vid] = weight_tons

with open('verificacion_caso3.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow([
        'VehicleId','LoadCap','FuelCap','RouteSequence','Municipalities','DemandSatisfied',
        'InitLoad','InitFuel','RefuelStops','RefuelAmounts','TollsVisited','TollCosts','VehicleWeights',
        'Distance','Time','FuelCost','TollCost','TotalCost'
    ])

    for v in ampl.getSet('V'):
        usage = z[v].value()
        if usage > 0.001:
            print(f"\n🛻 Vehículo {v}: Utilizado al {usage*100:.2f}%")

            # 🔗 Recolectar arcos
            arcs = []
            for i in ampl.getSet('N'):
                for j in ampl.getSet('N'):
                    if i != j and x[v, i, j].value() > 0.01:
                        arcs.append((i, j))

            # 🛣️ Reconstruir ruta secuencial
            route = ['PTO01']
            current = 'PTO01'
            visited = set(route)
            while True:
                next_nodes = [j for i, j in arcs if i == current and j not in visited]
                if next_nodes:
                    next_node = next_nodes[0]
                    route.append(next_node)
                    visited.add(next_node)
                    current = next_node
                else:
                    depot_return = [j for i, j in arcs if i == current and j == 'PTO01']
                    if depot_return and 'PTO01' not in route[1:]:
                        route.append('PTO01')
                    break

            print("🛣️ Ruta completa:", ' -> '.join(route))

            # Municipios visitados y demandas
            muni_visited = [n for n in route if n.startswith('MUN')]
            demand_satisfied = len(muni_visited)
            
            # Detalles de demanda por municipio
            demand_details = []
            for m in N:
                if m in muni_visited:
                    demand_details.append(f"{int(ampl.getParameter('demand')[m])}")
                elif m.startswith('MUN'):
                    demand_details.append("0")
                elif m.startswith('EST') or m.startswith('PEA') or m.startswith('PTO'):
                    demand_details.append("0")
            
            init_load = sum(ampl.getParameter('demand')[m] for m in muni_visited)
            load_cap = ampl.getParameter('Q')[v]
            fuel_cap = ampl.getParameter('F_cap')[v]

            # Distancia total
            total_distance = sum(
                dist[(route[idx], route[idx+1])]
                for idx in range(len(route)-1)
                if (route[idx], route[idx+1]) in dist
            )

            # Tiempo
            time = (total_distance / 40) * 60  # minutos

            # Recargas
            refuels = []
            refuel_amts = []
            fuel_cost = 0
            for est in E:
                amount = r[v, est].value()
                if amount > 0.01:
                    refuels.append(est)
                    refuel_amts.append(f"{amount:.0f}")
                    fuel_cost += amount * station_prices.get(est, 0)
                    print(f"⛽ {est}: {amount:.2f} gal x {station_prices[est]:.0f} = {amount*station_prices[est]:.0f}")

            print(f"💰 Costo combustible vehículo {v}: {fuel_cost:.0f}")

            # Peajes visitados
            tolls_visited = []
            toll_costs = []
            total_toll_cost = 0
            for t in T:
                if toll_used[v, t].value() > 0.01:
                    tolls_visited.append(t)
                    base_rate = toll_base_rates.get(t, 0)
                    rate_per_ton = toll_rates_per_ton.get(t, 0)
                    weight = vehicle_weights.get(v, 0)
                    toll_cost = base_rate + (rate_per_ton * weight)
                    toll_costs.append(f"{toll_cost:.0f}")
                    total_toll_cost += toll_cost
                    print(f"🛣️ {t}: {base_rate:.0f} + {rate_per_ton:.0f} x {weight:.1f}t = {toll_cost:.0f}")

            print(f"💰 Costo peajes vehículo {v}: {total_toll_cost:.0f}")

            # Costos de distancia del vehículo
            distance_cost = sum(
                (ampl.getParameter('Ft').value() + ampl.getParameter('Cm').value()) * dist[(i,j)] * x[v, i, j].value()
                for i, j in dist.keys() if x[v, i, j].value() > 0.01
            )

            total_cost_vehicle = distance_cost + fuel_cost + total_toll_cost
            print(f"💰 Costo total vehículo {v}: {total_cost_vehicle:.0f}")

            writer.writerow([
                v,
                f"{load_cap:.0f}",
                f"{fuel_cap:.0f}",
                '-'.join(route),
                '-'.join(muni_visited),
                '-'.join(demand_details),
                f"{init_load:.0f}",
                f"{fuel_cap:.0f}",
                len(refuels),
                '-'.join(refuel_amts) if refuel_amts else '0',
                len(tolls_visited),
                '-'.join(toll_costs) if toll_costs else '0',
                '-'.join([f"{vehicle_weights.get(v, 0):.0f}" for _ in muni_visited]) if muni_visited else '0',
                f"{total_distance:.1f}",
                f"{time:.1f}",
                f"{fuel_cost:.0f}",
                f"{total_toll_cost:.0f}",
                f"{total_cost_vehicle:.0f}"
            ])

print("✅ Archivo de verificación generado correctamente.")

✅ MustPassThrough generado con 76 entradas.
✅ Archivo vrp_data.dat creado correctamente!
🚀 Resolviendo con AMPL + Gurobi (límite de 10 minutos)...
Gurobi 12.0.1:   lim:time = 600
Gurobi 12.0.1: infeasible problem
330 simplex iterations
1 branching node

suffix dunbdd OUT;
✅ Costo total global: 0.0
✅ Archivo de verificación generado correctamente.
